# Feature Engineering

In [37]:
#Ładowanie bibliotek
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


import contractions
from symspellpy import SymSpell, Verbosity
from nrclex import NRCLex


In [19]:
#Podział danych na treningowe, testowe i walidacyjne

#df=pd.read_csv("YoutubeCommentsDataSet.csv")
#X = df.drop(columns=["Sentiment"])
#y = df['Sentiment']

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)
#y_test = pd.DataFrame(y_test, columns=["Sentiment"])


#ValidationData = pd.concat([X_val, y_val], axis=1)
#ValidationData.to_csv("ValidationData.csv", index=False)
#TestData = pd.concat([X_test, y_test], axis=1)
#TestData.to_csv("TestData.csv",index=False)
#TrainData = pd.concat([X_train, y_train], axis=1)
#TrainData.to_csv("TrainData.csv",index=False)


#print(f'Train shape: {X_train.shape}, Test shape: {X_test.shape}, Validation shape: {X_val.shape}')

Train shape: (9019, 1), Test shape: (3866, 1), Validation shape: (5523, 1)


In [53]:
#Zaczynam tworzyć klasy do pipelina

class CleanCommentTransform(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Inicjalizacja lematyzatora i stopwords
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
    def fit(self, X, y=None):
        # Fit nie wykonuje żadnych operacji w tym przypadku
        return self
    
    def transform(self, X):
        # Tworzenie kopii danych, aby nie zmieniać oryginalnych
        df = X.copy()
        df['Comment'] = df['Comment'].fillna('')
        
        def preprocess_text(text):
            # Usuwanie interpunkcji
            text = text.lower()
            text = ''.join([char for char in text if char not in string.punctuation])
            
            # Tokenizacja
            tokens = word_tokenize(text)
            
            # Lematyzacja i usuwanie stopwords
            tokens = [self.lemmatizer.lemmatize(word) for word in tokens if word not in self.stop_words]
            
            return " ".join(tokens)
        
        # Zastosowanie transformacji na kolumnie 'Comment'
        df['Clean_Comment'] = df['Comment'].apply(preprocess_text)
        
        return df[['Clean_Comment']].to_numpy()


In [61]:

pipeline = Pipeline([
    ('clean_comment', CleanCommentTransform())
])

# Przykładowe dane wejściowe
data = pd.DataFrame({
    'Comment': [
        'This is a test comment!',
        'Another test, with more words.',
        'Testing the lemmatizer and stopwords removal.',
        None
    ],
    'Sentiment': [1, 0, 1,0]
})

# Przetwarzanie danych przez pipeline
processed_data = pipeline.fit_transform(data)

# Wyświetlanie przetworzonych danych
print(processed_data)

[['test comment']
 ['another test word']
 ['testing lemmatizer stopwords removal']
 ['']]
